In [1]:
# Célula 1: Inicialização do Spark
import sys
import os

# Adiciona o diretório src ao path
src_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))
if src_path not in sys.path:
    sys.path.insert(0, src_path)

print(f"📂 Diretório atual: {os.getcwd()}")
print(f"📂 Diretório src: {src_path}")

# Importa e inicializa o Spark
from db_connection import get_spark_session

spark = get_spark_session(app_name="ETL_Project")

# Teste básico
print("\n🔹 Teste básico do Spark:")
test_data = [(1, "teste"), (2, "spark"), (3, "funcionando")]
test_df = spark.createDataFrame(test_data, ["id", "mensagem"])
test_df.show()

print("\n✅ Spark pronto para uso!")

📂 Diretório atual: C:\Users\Gabriel\Documents\ml-data-engineering-project\notebooks
📂 Diretório src: C:\Users\Gabriel\Documents\ml-data-engineering-project\src
🔹 Inicializando Spark com driver JDBC...
🔹 Driver configurado: C:\Users\Gabriel\Documents\ml-data-engineering-project\driver\mssql-jdbc-13.2.1.jre11.jar
✅ Spark inicializado com sucesso!
✅ Spark Version: 4.0.1
✅ Spark UI: http://localhost:4040

🔹 Teste básico do Spark:
+---+-----------+
| id|   mensagem|
+---+-----------+
|  1|      teste|
|  2|      spark|
|  3|funcionando|
+---+-----------+


✅ Spark pronto para uso!


In [2]:
# Teste de Conexão com Banco de Dados

from db_connection import get_jdbc_params

print("🔹 Testando conexão com o banco de dados...\n")

# Obtém parâmetros de conexão
jdbc_params = get_jdbc_params()

# Tenta conectar e listar tabelas do banco
try:
    # Query para listar tabelas (SQL Server)
    query = """
        (SELECT TABLE_SCHEMA, TABLE_NAME, TABLE_TYPE 
         FROM INFORMATION_SCHEMA.TABLES 
         WHERE TABLE_TYPE = 'BASE TABLE') AS tables
    """
    
    df_tables = spark.read.jdbc(
        url=jdbc_params["url"],
        table=query,
        properties={
            "user": jdbc_params["user"],
            "password": jdbc_params["password"],
            "driver": jdbc_params["driver"]
        }
    )
    
    print("✅ Conexão com banco de dados estabelecida com sucesso!\n")
    print(f"📊 Total de tabelas encontradas: {df_tables.count()}\n")
    print("🔹 Tabelas disponíveis:")
    df_tables.show(20, truncate=False)
    
except Exception as e:
    print(f"❌ Erro ao conectar com o banco de dados:")
    print(f"   {str(e)}\n")
    print("💡 Verifique:")
    print("   - Se o arquivo .env está configurado corretamente")
    print("   - Se o servidor SQL Server está rodando")
    print("   - Se as credenciais estão corretas")

🔹 Testando conexão com o banco de dados...

✅ Conexão com banco de dados estabelecida com sucesso!

📊 Total de tabelas encontradas: 6

🔹 Tabelas disponíveis:
+------------+--------------+----------+
|TABLE_SCHEMA|TABLE_NAME    |TABLE_TYPE|
+------------+--------------+----------+
|dbo         |example_clean |BASE TABLE|
|dbo         |categorias    |BASE TABLE|
|dbo         |produtos      |BASE TABLE|
|dbo         |clientes      |BASE TABLE|
|dbo         |vendas        |BASE TABLE|
|dbo         |covid19_brazil|BASE TABLE|
+------------+--------------+----------+



In [3]:
# Lendo o arquivo example.csv

csv_path = "../input/csv/example.csv"

try:
    df = spark.read.csv(csv_path, header=True, inferSchema=True)
    print("✅ CSV lido com sucesso!")
    df.show(5)
except Exception as e:
    print("❌ Erro ao ler o CSV:", e)


✅ CSV lido com sucesso!
+---+--------+-----+
| id|    nome|idade|
+---+--------+-----+
|  1| gabriel|   18|
|  2|    joao|   20|
|  3|patricia|   20|
|  4|  raquel|   18|
|  5|    eloa|   18|
+---+--------+-----+



In [4]:
# Importações para funcionara o uso das utils de src

import sys
import os

# Caminho absoluto da raiz do projeto (uma pasta acima do notebooks/)
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
src_path = os.path.join(project_root, "src")

# Adiciona src ao sys.path
if src_path not in sys.path:
    sys.path.append(src_path)

# Agora importa normalmente
from utils import create_logger
from db_connection import get_spark_session, get_jdbc_params

print("✅ Módulos importados com sucesso!")


✅ Módulos importados com sucesso!


In [5]:
# Tratar dados e salvar
import os

output_path = "../output/clean"
os.makedirs(output_path, exist_ok=True)

log = create_logger()

# Exemplo de tratamento: remover duplicatas e nulos
df_clean = df.dropDuplicates().dropna()

output_file = os.path.join(output_path, "example-clean.csv")
df_clean.coalesce(1).write.mode("overwrite").option("header", True).csv(output_file)

log("Arquivo tratado salvo em example-clean.csv")

Arquivo tratado salvo em example-clean.csv


In [6]:
# Testar leitura do .env e logar variáveis

from dotenv import load_dotenv
import os

load_dotenv()

print("DB_URL:", os.getenv("DB_URL"))
print("DB_USER:", os.getenv("DB_USER"))

DB_URL: jdbc:sqlserver://localhost:1434;databaseName=datalake_local;encrypt=true;trustServerCertificate=true
DB_USER: sa


In [7]:
spark.stop()